In [1]:
import pandas as pd
import openpyxl
import tkinter.filedialog
import datetime
import msvcrt
from IPython.display import display

### Input new bills records and appended to the history

In [3]:
def strip_in_data(data):  # Remove leading and trailing spaces from column names and data.
    data = data.rename(columns={column_name: column_name.strip() for column_name in data.columns})
    data = data.applymap(lambda x: x.strip().strip('¥') if isinstance(x, str) else x)
    return data


def read_data_wx(path):  # 获取微信数据
    d_wx = pd.read_csv(path, header=16, skipfooter=0, encoding='utf-8')  # 数据获取，微信
    d_wx = d_wx.iloc[:, [0, 4, 7, 1, 2, 3, 5]]  # 按顺序提取所需列
    d_wx = strip_in_data(d_wx)  # 去除列名与数值中的空格。
    d_wx.iloc[:, 0] = d_wx.iloc[:, 0].astype('datetime64')  # 数据类型更改
    d_wx.iloc[:, 6] = d_wx.iloc[:, 6].astype('float64')  # 数据类型更改
    d_wx = d_wx.drop(d_wx[d_wx['收/支'] == '/'].index)  # 删除'收/支'为'/'的行
    d_wx.rename(columns={'当前状态': '支付状态', '交易类型': '类型', '金额(元)': '金额'}, inplace=True)  # 修改列名称
    d_wx.insert(1, '来源', "微信", allow_duplicates=True)  # 添加微信来源标识
    len1 = len(d_wx)
    print("Sucessfully Read " + str(len1) + " 「Wechat」bills\n")
    return d_wx


def read_data_zfb(path):  # 获取支付宝数据
    d_zfb = pd.read_csv(path, header=4, skipfooter=7, encoding='gbk')  # 数据获取，支付宝
    d_zfb = d_zfb.iloc[:, [2, 10, 11, 6, 7, 8, 9]]  # 按顺序提取所需列
    d_zfb = strip_in_data(d_zfb)  # 去除列名与数值中的空格。
    d_zfb.iloc[:, 0] = d_zfb.iloc[:, 0].astype('datetime64')  # 数据类型更改
    d_zfb.iloc[:, 6] = d_zfb.iloc[:, 6].astype('float64')  # 数据类型更改
    d_zfb = d_zfb.drop(d_zfb[d_zfb['收/支'] == ''].index)  # 删除'收/支'为空的行
    d_zfb.rename(columns={'交易创建时间': '交易时间', '交易状态': '支付状态', '商品名称': '商品', '金额（元）': '金额'}, inplace=True)  # 修改列名称
    d_zfb.insert(1, '来源', "支付宝", allow_duplicates=True)  # 添加支付宝来源标识
    len2 = len(d_zfb)
    print("Sucessfully Read " + str(len2) + " 「Alipay」bills\n")
    return d_zfb


def add_cols(data):  # 增加3列数据
    # 逻辑1：取值-1 or 1。-1表示支出，1表示收入。
    data.insert(8, '逻辑1', -1, allow_duplicates=True)  # 插入列，默认值为-1
    for index in range(len(data.iloc[:, 2])):  # 遍历第3列的值，判断为收入，则改'逻辑1'为1
        if data.iloc[index, 2] == '收入':
            data.iloc[index, 8] = 1

        # update 2021/12/29: 修复支付宝理财收支逻辑bug
        elif data.iloc[index, 5] == '蚂蚁财富-蚂蚁（杭州）基金销售有限公司' and '卖出' in data.iloc[index, 6]:
            data.iloc[index, 8] = 1
        elif data.iloc[index, 5] == '蚂蚁财富-蚂蚁（杭州）基金销售有限公司' and '转换至' in data.iloc[index, 6]:
            data.iloc[index, 8] = 0
        elif data.iloc[index, 2] == '其他' and '收益发放' in data.iloc[index, 6]:
            data.iloc[index, 8] = 1
        elif data.iloc[index, 2] == '其他' and '现金分红' in data.iloc[index, 6]:
            data.iloc[index, 8] = 1
        elif data.iloc[index, 2] == '其他' and '买入' in data.iloc[index, 6]:
            data.iloc[index, 8] = -1
        elif data.iloc[index, 2] == '其他':
            data.iloc[index, 8] = 0

    # 逻辑2：取值0 or 1。1表示计入，0表示不计入。
    data.insert(9, '逻辑2', 1, allow_duplicates=True)  # 插入列，默认值为1
    for index in range(len(data.iloc[:, 3])):  # 遍历第4列的值，判断为资金流动，则改'逻辑2'为0
        col3 = data.iloc[index, 3]
        if (col3 == '提现已到账') or (col3 == '已全额退款') or (col3 == '已退款') or (col3 == '退款成功') or (col3 == '还款成功') or (
                col3 == '交易关闭'):
            data.iloc[index, 9] = 0

    # 月份
    data.insert(1, '月份', 0, allow_duplicates=True)  # 插入列，默认值为0
    for index in range(len(data.iloc[:, 0])):
        time = data.iloc[index, 0]
        data.iloc[index, 1] = time.month  # 访问月份属性的值，赋给这月份列

    # 乘后金额
    data.insert(11, '乘后金额', 0, allow_duplicates=True)  # 插入列，默认值为0
    for index in range(len(data.iloc[:, 8])):
        money = data.iloc[index, 8] * data.iloc[index, 9] * data.iloc[index, 10]
        data.iloc[index, 11] = money
    return data

### Here is an example, and package this part into `DataHandler.py`

In [4]:
# New bills
filename = r'New_records_alipay.csv'
path = r'D:\Projects\Accounts\Dataloader'
data_zfb = read_data_zfb(path + '\\' + filename) # New bills
_new_part = add_cols(data_zfb)
_new_part.to_csv(path + '\\' + 'New_records_alipay.csv')

new_part = pd.read_csv(path + '\\' + 'New_records_alipay.csv', index_col=0).fillna('')


Sucessfully Read 148 「Alipay」bills



C:\Users\Administrator\AppData\Local\Temp\ipykernel_8328\3605182696.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  d_zfb = pd.read_csv(path, header=4, skipfooter=7, encoding='gbk')  # 数据获取，支付宝


In [5]:
new_part

,交易时间,月份,来源,收/支,支付状态,类型,交易对方,商品,金额,逻辑1,逻辑2,乘后金额
0,2023-12-10 18:17:55,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,商城路--三林东,4.00,-1,1,-4.00
1,2023-12-10 14:17:04,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,马当路--商城路,3.00,-1,1,-3.00
2,2023-12-10 13:21:41,12,支付宝,不计收支,交易成功,即时到账交易,美团,大众点评订单-23121011100400000021574530283011,5.90,-1,1,-5.90
3,2023-12-10 13:13:45,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,三林东--西藏南路,4.00,-1,1,-4.00
4,2023-12-09 20:16:12,12,支付宝,支出,交易成功,即时到账交易,Valve,S2P1172140557 Steam Purchase,15.80,-1,1,-15.80
...,...,...,...,...,...,...,...,...,...,...,...,...
143,2023-11-10 13:21:59,11,支付宝,不计收支,交易成功,即时到账交易,高德打车,高德地图打车订单,13.49,-1,1,-13.49
144,2023-11-10 12:22:55,11,支付宝,不计收支,交易成功,即时到账交易,高德打车,高德地图打车订单,12.59,-1,1,-12.59
145,2023-11-10 11:58:55,11,支付宝,不计收支,交易成功,即时到账交易,湘湖荷花餐厅,美团收银909700186304508484,3.49,-1,1,-3.49
146,2023-11-10 10:13:56,11,支付宝,不计收支,交易成功,即时到账交易,罗森(湘湖金融小镇店),支付宝支付湘湖金融小镇店消费,5.90,-1,1,-5.90


In [6]:
# Historic bills
his_path = r'D:\Projects\Accounts\Dataloader\his\History_records_alipay.csv'
his_part = pd.read_csv(his_path, index_col=0).fillna('') # New bills

In [7]:
his_part.sort_values('交易时间')

,交易时间,月份,来源,收/支,支付状态,类型,交易对方,商品,金额,逻辑1,逻辑2,乘后金额
4204,2020-12-01 10:31:33,12,支付宝,不计收支,交易成功,即时到账交易,蚂蚁财富-蚂蚁（杭州）基金销售有限公司,蚂蚁财富-天弘中证银行指数A-买入,23.0,-1,1,-23.0
4203,2020-12-01 13:32:39,12,支付宝,不计收支,交易成功,支付宝担保交易,ellen0wang,基础综合英语 学生用书 研究生英语教材 pdf版电子书,5.5,-1,1,-5.5
4202,2020-12-01 13:37:35,12,支付宝,支出,交易成功,即时到账交易,App Store & Apple Music,App Store & Apple Music: 于 11.30完成的购买,3.0,-1,1,-3.0
4201,2020-12-01 21:53:47,12,支付宝,不计收支,交易成功,即时到账交易,麦当劳(深南西路分店餐厅),麦当劳餐厅(深圳) 有限公司,6.0,-1,1,-6.0
4200,2020-12-01 21:54:36,12,支付宝,不计收支,交易成功,即时到账交易,麦当劳(深南西路分店餐厅),麦当劳餐厅(深圳) 有限公司,0.1,-1,1,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-12-07 19:29:46,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,静安寺--三林东,4.0,-1,1,-4.0
3,2023-12-07 19:40:18,12,支付宝,不计收支,交易成功,即时到账交易,上海醉食汇,上海醉食汇安盛街店,4.2,-1,1,-4.2
2,2023-12-08 08:39:57,12,支付宝,不计收支,交易成功,即时到账交易,麦当劳,上海金拱门食品有限公司,13.5,-1,1,-13.5
1,2023-12-08 08:49:23,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,三林东--静安寺,4.0,-1,1,-4.0


In [8]:
data_merge = pd.concat([his_part,new_part]).drop_duplicates()

In [9]:
data_merge.sort_values('交易时间')

,交易时间,月份,来源,收/支,支付状态,类型,交易对方,商品,金额,逻辑1,逻辑2,乘后金额
4204,2020-12-01 10:31:33,12,支付宝,不计收支,交易成功,即时到账交易,蚂蚁财富-蚂蚁（杭州）基金销售有限公司,蚂蚁财富-天弘中证银行指数A-买入,23.0,-1,1,-23.0
4203,2020-12-01 13:32:39,12,支付宝,不计收支,交易成功,支付宝担保交易,ellen0wang,基础综合英语 学生用书 研究生英语教材 pdf版电子书,5.5,-1,1,-5.5
4202,2020-12-01 13:37:35,12,支付宝,支出,交易成功,即时到账交易,App Store & Apple Music,App Store & Apple Music: 于 11.30完成的购买,3.0,-1,1,-3.0
4201,2020-12-01 21:53:47,12,支付宝,不计收支,交易成功,即时到账交易,麦当劳(深南西路分店餐厅),麦当劳餐厅(深圳) 有限公司,6.0,-1,1,-6.0
4200,2020-12-01 21:54:36,12,支付宝,不计收支,交易成功,即时到账交易,麦当劳(深南西路分店餐厅),麦当劳餐厅(深圳) 有限公司,0.1,-1,1,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,2023-12-09 20:16:12,12,支付宝,支出,交易成功,即时到账交易,Valve,S2P1172140557 Steam Purchase,15.8,-1,1,-15.8
3,2023-12-10 13:13:45,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,三林东--西藏南路,4.0,-1,1,-4.0
2,2023-12-10 13:21:41,12,支付宝,不计收支,交易成功,即时到账交易,美团,大众点评订单-23121011100400000021574530283011,5.9,-1,1,-5.9
1,2023-12-10 14:17:04,12,支付宝,不计收支,交易成功,即时到账交易,上海都畅数字技术有限公司,马当路--商城路,3.0,-1,1,-3.0


In [10]:
data_merge.to_csv(r'D:\Projects\Accounts\Dataloader\his\History_records_alipay.csv')

### Analysis: Daily expense is the most available way to quantify behaviours. Here is the following steps:
1. Create a set that useful to collect your expense.
2. Categorize the expense by rows.
3. Do your individual analysis with some visualization.

#### My set (this part would be the key, it reflects your value system.
['必要性食品','非必要性食品','交通','一般耐用品','电子耐用品','订阅类信息服务或娱乐产品','线下娱乐或休闲','其他']

#### GPT-4.0 works.
- OpenAI's GPT4 API works in this situation.
- Time & Financial cost almost unavailable but still ok.

In [4]:
# read api key
with open(r'D:\accounts_key.txt','r') as key_file:
    OPENAI_API_KEY = key_file.readline()

import os
import openai 

openai.api_key = OPENAI_API_KEY

In [5]:
# read bills record
bill_df = pd.read_csv(r'D:\Projects\Accounts\Dataloader\his\History_records_alipay.csv', index_col=0)

In [26]:
# Bill
class Bill(object):
    def __init__(
        self,
        row:pd.core.series.Series,
        ):
        '''
        Categorization by row.
        '''
        self.row = row
        self.txt_eles = row.loc[['交易时间','交易对方','商品','金额']].values
        self.head_txts = '这是我的交易记录:' 
        self.txts = f'在{self.txt_eles[0]}, 我花费了{self.txt_eles[-1] }元人民币向{self.txt_eles[1]} 购买了{self.txt_eles[2]}。'
        self.tail_txts = \
        '''请帮我将这笔订单记录分类（只返回类别名称，如果难以判断则归类为“其他”），在以下类别 ['必要性食品','非必要性食品','交通','一般耐用品','电子耐用品','订阅类信息服务或娱乐产品','线下娱乐或休闲','其他'] 中，上面这个订单属于哪一类?'''
        
        self.asking_texts = self.head_txts + self.txts + self.tail_txts
    
    # classifier
    def classify_expense(self):
        # try:
        response = openai.ChatCompletion.create(
            model='gpt-4',#"gpt-4.0",
            messages=[{"role": "system", "content": "你是一个根据订单记录对订单进行分类的智能助力，你言简意赅。你擅长结合交易对方的名称，交易发生的时间，交易金额判断支出类型。"},
                    {"role": "user", "content": self.asking_texts}],
            max_tokens=32
        )
        return response['choices'][0]['message']['content']
        # except Exception as e:
        #     print(f"Error: {e}")
        #     return "Categorization Failed!."

In [27]:
# Bills_df = bill_df.set_index('交易时间')
Bills_df = bill_df.sort_values('交易时间')

In [28]:
cates = {}
for idx, rcd in Bills_df.iterrows():
    one_bill = Bill(rcd)
    cates[idx] = one_bill.classify_expense()
    # break


KeyboardInterrupt: 

In [29]:
cates

{4204: '根据您提供的订单信息，这笔订单属于“一般耐用品”类别。',
 4203: '根据提供的订单记录，这笔订单属于"订阅类信息服务或娱乐产品"',
 4202: '根据交易对方的名称为App Store & Apple Music，交易发生的时间为11月30日，交易金额为'}

#### For free version, we try using LLAMA-2-70B.
- Bad classifying perform.
- Same problem with GPT-3.5-turbo, usually return too much more than I ask for.
- Sometimes successfully limit the returns.

In [37]:
import os
import replicate

In [88]:
output = replicate.run(
  "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
  input={
    "debug": False,
    "top_k": 50,
    "top_p": 1,
    "prompt": "“12:41:20在上海都畅数字技术有限公司购买了三林东--西藏南路”。 For this expense record text, what would you categorize it to? 'food','sanck','transportation','durable goods','electronic'or'subscription-based service' or 'offline entertainment'? Don't say any unnecessary, you must give me ONE WORD of those categories.",
    "temperature": 1,
    "system_prompt": "You are a good assistant good at categorizing expense base on chinese texts.",
    "max_new_tokens": 16,
    "min_new_tokens": 2
  })

In [89]:
for item in output:
    print(item, end="")

 " Durable goods"